# Грузим библиотеки

In [50]:
from flask import Flask
from flask import request
import requests
from flask import make_response
import os
import json
from pandas import DataFrame


from methods import access
from methods.emoji import emoji
from methods import telegram_bot_methods
from methods import google_maps_methods
from methods import psql_methods
from methods import reply




token = access.token()
api = access.api()

#bot = telebot.TeleBot(token)

application = Flask(__name__)  # Change assignment here




# создаем webhook
@application.route("/set_webhook")
def webhook():
    url = api + token + '/setWebhook'
    params = {'url' : 'https://fin-budg-bot.herokuapp.com/main'
    }
    r = requests.post(url,
                      json=params)
    return "!", 200



# тестовый вывод
@application.route("/")  
def hello():
    return "Hello World!"




# запуск основной функции
@application.route('/main', methods=['GET', 'POST'])
def main():
    try:
        json_update = json.loads(request.get_data())


        #Изначально для отправки кнопки пустые
        reply_markup = None

        #получаем id чата и текст сообщения
        chat_id = json_update['message']['chat']['id']
        command = json_update['message']['text']

        #в зависимости от команды выбираем действие
        if command[0] != '/':
            text = 'Не понимаю... Помочь? /help'

        else:
            if 'start' in command:
                r = psql_methods.new_user(chat_id , json_update)
                text = emoji('фанфары') + 'Добрый день! \n' 
                text += 'Я планирую домашний бюджет и напоминаю об операциях в течение месяца. \n'
                text += 'Я развиваюсь в свободное время, текущие команды можно увидеть в меню /help. \n'
                text += 'Либо сразу заведите свой кошелек и забудьте о том, чтобы держать бюджет семьи в голове!' + emoji('банкноты')
                reply_markup = {'keyboard': [['/help'],['/make_wallet']], 'resize_keyboard': True, 'one_time_keyboard': False}
            elif 'help' in command:
                r = psql_methods.last_state(chat_id,command)
                text = 'Привет!' +  emoji('фанфары') + '\n'
                text += 'Задачи кошелька: \n'
                text += '/make_wallet - создание кошелька \n'
                text += '/expense_add - добавить трату \n'
                text += '/expense_update - изменить/удалить трату \n'
                text += '/wallet_report - отчет по операциям \n'
                text += '/wallet_advice - совет'
            elif 'make_wallet' in command:
                r = psql_methods.last_state(chat_id,command)
                r = psql_methods.make_wallet(chat_id)
                text = r['text']
                reply_markup = r['reply_markup']
            elif 'wallet_action' in command:
                r = psql_methods.last_state(chat_id,command)
                text = 'Что нужно сделать с личным кошельком? \n'
                text += '/expense_add - добавить фактическую операцию'
                reply_markup = {'keyboard': [['/expense_add']], 'resize_keyboard': True, 'one_time_keyboard': False}
            elif 'expense_add' in command:
                r = psql_methods.last_state(chat_id,command)
                text = 'Выберите тип траты'
                keyboard_expense = reply.keyboard_expense()
                reply_markup = {'keyboard': keyboard_expense, 'resize_keyboard': True, 'one_time_keyboard': True}

            keyboard_expense = reply.keyboard_expense()
            last_state = psql_methods.current_last_state(chat_id)
            #если пришел запрос на добавление траты по типу - спрашиваем сумму
            elif command in keyboard_expense and last_state == '/expense_add':
                text = 'ок! Введите сумму операции'
                r = psql_methods.insert_state_info_1(chat_id = chat_id , state_info_one = command)










            else:
                text = 'Неизвестная команда, для списка команд выбирите команду /help'
  
        #отправляем сообщение
        send_result = telegram_bot_methods.send_message(chat_id = chat_id, text = text, reply_markup = reply_markup)




            
        return "!", 200
    except:

        return "!", 200







if __name__ == "__main__":
    port = int(os.getenv('PORT', 5000))
    application.run(debug=False, port=port, host='0.0.0.0')



SyntaxError: invalid syntax (<ipython-input-50-1a4b91487028>, line 104)

In [49]:
X = {'update_id': 201011794, 'message': {'message_id': 159, 'from': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'language_code': 'ru-RU'}, 'chat': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'type': 'private'}, 'date': 1502611993, 'text': '/help', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 5}]}}
X['message']['text'] = '/start'
X['message']['chat']['last_name']

'Толмачёв'

In [ ]:
import datetime
now = datetime.datetime.now()
str(now.year)+str(now.month if now.month >= 10 else  '0'+str(now.month))+str(now.day if now.day >= 10 else  '0'+str(now.hour)) +' '+str(now.hour if now.hour >= 10 else  '0'+str(now.hour)) + str(now.minute if now.minute >= 10 else  '0'+str(now.minute)) + str(now.second if now.second >= 10 else  '0'+str(now.second))

In [41]:
from methods import access
from methods import psql_methods

import psycopg2
from pandas import DataFrame
chat_id =84723474

PSQL_heroku_keys = access.PSQL_heroku_keys()

#создаем подключение к PSQL
conn = psycopg2.connect("dbname='%(dbname)s' port='%(port)s' user='%(user)s' host='%(host)s' password='%(password)s'" % PSQL_heroku_keys)
cur = conn.cursor()

cur.execute("SELECT current_state as id from public.state" )

df = DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

cur.close()

In [46]:
df['id'][0]


True

In [ ]:
from methods import access
from methods import psql_methods

r = psql_methods.make_wallet(84723474)



In [29]:
#тест отправки клавиатуры

from methods import access
import requests
import json

chat_id = 84723474
text = '123'

token = access.token()
api = access.api()

reply_markup = {'keyboard': [['1','2'],['2'],['4'],['5']], 'resize_keyboard': True, 'one_time_keyboard': True}
#reply_markup = json.dumps(reply_markup)



url = api + token + '/sendMessage'
params = {'chat_id' : chat_id
        ,'text' : text
          ,'reply_markup': reply_markup
          
}
r = requests.post(url,
                  json=params)

In [36]:
#тест отправки клавиатуры

from methods import access
import requests
import json

chat_id = 84723474
text = '123'

token = access.token()
api = access.api()

reply_markup = {'keyboard': [['1']]}
#reply_markup = json.dumps(reply_markup)



url = api + token + '/sendMessage'
params = {'chat_id' : chat_id
        ,'text' : text
          ,'reply_markup': reply_markup
          
}
r = requests.post(url,
                  json=params)